# **Data Generator Testing**
This notebook demonstrates the necessary imports and setup to test the Data Generator. If you follow along with this notebook then you can test the Data Generator with our Amazing Grace audio data. The batches are generated as individual elements because of the inefficiency in reshaping audio data, but the generator still considerably improves performance.

Use the following commands to install the two libraries. ffmpeg is not used at this time but may be necessary later. Librosa allows us to easily read and process audio data.

In [1]:
!pip install librosa


In [2]:
!pip install ffmpeg


  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6083 sha256=9101b7a01db7a2d03dadd99979dea16c283b49b7eea500ad9ea890b01134709e
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


Verify directory structure and create directory for audio data.

In [3]:
ls

sample_data/


In [4]:
mkdir grace_data

In [5]:
ls

grace_data/  sample_data/


In [6]:
cd grace_data/

/content/grace_data


Use the following two commands to unpack the Amazing Grace data from the below URL and into the current directory, creating a subdirectory under grace_data called amazing_grace.

In [7]:
!wget https://ccrma.stanford.edu/damp/performances/amazing_grace/amazing_grace.tar.gz

--2020-10-13 04:15:12--  https://ccrma.stanford.edu/damp/performances/amazing_grace/amazing_grace.tar.gz
Resolving ccrma.stanford.edu (ccrma.stanford.edu)... 171.64.197.141
Connecting to ccrma.stanford.edu (ccrma.stanford.edu)|171.64.197.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18757937250 (17G) [application/x-gzip]
Saving to: ‘amazing_grace.tar.gz’

amazing_grace.tar.g 100%[===================>]  17.47G  32.1MB/s    in 9m 55s  

2020-10-13 04:25:07 (30.1 MB/s) - ‘amazing_grace.tar.gz’ saved [18757937250/18757937250]



In [ ]:
!tar xvzf amazing_grace.tar.gz;

Verify directory structure and presence of data.

In [10]:
%ls

amazing_grace/     amazing_grace.midi    amazing_grace.tsv
amazing_grace.m4a  amazing_grace.tar.gz


In [11]:
cd amazing_grace

/content/grace_data/amazing_grace


In [ ]:
%ls

Import some more necessary libraries. If you are running this code locally it is assumed you have these installed but if not then use pip and install these popular libraries.

In [13]:
import numpy as np
import keras

import random
import librosa
from os import listdir
from os.path import isfile, join

Create list of files from the data and verify that list contains correct content.

In [14]:
file_list = [f for f in listdir(".") if isfile(join(".", f))]

In [15]:
cd ..

/content/grace_data


In [16]:
cd ..

/content


Below is the script for the Data Generator. It takes in parameters corresponding to the data and then generates batches of data upon call (our batch size is set to one since we are dealing with audio data as described above, in the future we may look into reshaping methods).

We are not generating any test data right now, as you can see with the commented out 'y' variable and corresponding code. This would generate labels along with the training data.

In [36]:
import numpy as np
import keras

import random
import librosa
from os import listdir
from os.path import isfile, join

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_prefix, list_IDs, labels, batch_size, dim, n_channels,
                 n_classes, shuffle):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.prefix = path_prefix
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self.__data_generation(list_IDs_temp) #,y when other stuff is uncommented

        return X#, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        
        curr_len = 0
        for i,ID in enumerate(list_IDs_temp):
          print(ID)
          curr_len = len(self.load_audio('grace_data/amazing_grace/'+ID))
        print(curr_len)
        X = np.empty((self.batch_size, curr_len))
        #y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #self.load_audio('grace_data/amazing_grace/312087870_108215812.m4a')
            X[i,] = self.load_audio(self.prefix + "/" + ID)

            # Store class
            #y[i] = self.labels[ID]

        return X#, keras.utils.to_categorical(y, num_classes=self.n_classes)

    def load_audio(self,audio_file_path):
        """Load audio to numpy array and return it
        """
        x,sr = librosa.load(audio_file_path, sr = None)
        return x

In [37]:
print ("Number of audio files: " + str(len(file_list)))

Number of audio files: 17582


Construct a data generator with parameters corresponding to data, set labels to just one for now since we are not performing any classification at this moment.

In [38]:
dg = DataGenerator("grace_data/amazing_grace",file_list,["grace"]*len(file_list),1,(1,len(file_list)), 1,1,True)

Verify our loading of audio to a numpy array works by calling the function and checking the length.

In [39]:
dg.load_audio('grace_data/amazing_grace/312087870_108215812.m4a')

array([-1.2207031e-04, -1.2817383e-03, -2.0141602e-03, ...,
        3.0517578e-05,  0.0000000e+00,  3.0517578e-05], dtype=float32)

In [40]:
len(dg.load_audio('grace_data/amazing_grace/312087870_108215812.m4a'))

4215424

In [41]:
len(dg.load_audio('grace_data/amazing_grace/510603659_264818125.m4a'))

6718912

Finally, here is where we test our data generator. Call the __getitem__ method, the parameter which you pass is an index which corresponds to the batch you want to generate. For example, passing in 0 would mean you want the first batch of audio files' data.

In [42]:
dg.__getitem__(0)

315369988_100070961.m4a
4215168


array([[-1.95312500e-03, -1.73950195e-03,  2.22778320e-03, ...,
        -9.15527344e-05,  1.22070312e-04,  9.15527344e-05]])

In [43]:
dg.__getitem__(1)

186214898_38494794.m4a
8389952


array([[-0.00061035, -0.00057983, -0.00059509, ...,  0.00109863,
         0.00105286,  0.00100708]])

In [44]:
dg.__getitem__(200)

340056358_108372264.m4a
9110592


array([[ 1.22070312e-04,  2.74658203e-04,  1.52587891e-04, ...,
        -3.05175781e-05,  1.83105469e-04,  1.83105469e-04]])

In [45]:
dg.__getitem__(3102)

115538407_72955355.m4a
4214912


array([[-0.00027466, -0.00054932,  0.00100708, ...,  0.        ,
         0.        ,  0.        ]])

In [46]:
dg.__len__()

17582

In [47]:
dg.on_epoch_end()